<a href="https://colab.research.google.com/github/nariba/hello-pycuda/blob/main/CPUvsGPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip3 install pycuda

     |████████████████████████████████| 1.6MB 13.2MB/s 
     |████████████████████████████████| 71kB 9.9MB/s 
     |████████████████████████████████| 81kB 11.0MB/s 
  Created wheel for pycuda: filename=pycuda-2020.1-cp36-cp36m-linux_x86_64.whl size=620887 sha256=d5cd7609af96cfd03a02a715a3fa42cd1f89ab83fb5a053a67bde61cda7e3528
  Stored in directory: /root/.cache/pip/wheels/8f/78/d1/5bb826f81d9d490297a348d818ff3ee6dd6f2075b06dde6ea0
  Created wheel for pytools: filename=pytools-2020.4.3-py2.py3-none-any.whl size=61374 sha256=dcc99f81f75e7941fd9e2b7a8bd17f00581c2567761b6d0de0a2c5dee32deaba
  Stored in directory: /root/.cache/pip/wheels/af/c7/81/a22edb90b0b09a880468b2253bb1df8e9f503337ee15432c64
Successfully built pycuda pytools


In [2]:
import os

In [3]:
os.mkdir("cuda")

In [4]:
import math
import numpy
import time
import pycuda.gpuarray
from pycuda.compiler import SourceModule

In [5]:
import pycuda.autoinit

In [6]:
cuda_file_path = os.path.abspath("./cuda")

In [7]:
module = SourceModule("""
#include "kernel_functions_for_math_1d.cu"
""", include_dirs=[cuda_file_path])

In [8]:
plus_one_kernel = module.get_function("plus_one_kernel")

In [9]:
num_components = numpy.int32(10)
x = numpy.arange(num_components, dtype=numpy.int32)

In [10]:
x_gpu = pycuda.gpuarray.to_gpu(x)
y_gpu = pycuda.gpuarray.zeros(num_components, dtype=numpy.int32)

In [11]:
threads_per_block = (256, 1, 1)
blocks_per_grid = (math.ceil(num_components / threads_per_block[0]), 1, 1)

In [12]:
plus_one_kernel(num_components, y_gpu, x_gpu, block=threads_per_block, grid=blocks_per_grid)

In [13]:
y_gpu.get()

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10], dtype=int32)

In [14]:
# Measure CPU time
time_start_cpu = time.time()
x = x + 1
time_end_cpu = time.time()

In [15]:
print("CPU calculation {0} [msec]".format(1000 * (time_end_cpu - time_start_cpu)))

CPU calculation 0.08797645568847656 [msec]


In [16]:
# Measure GPU time
import pycuda.driver

In [17]:
time_start_gpu = pycuda.driver.Event()
time_end_gpu = pycuda.driver.Event()

In [18]:
time_start_gpu.record()
plus_one_kernel(num_components, y_gpu, x_gpu, block=threads_per_block, grid=blocks_per_grid)
time_end_gpu.record()
time_end_gpu.synchronize()

In [19]:
print("GPU calculation {0} [msec]".format(time_start_gpu.time_till(time_end_gpu)))

GPU calculation 0.1987839937210083 [msec]


In [20]:
num_components = numpy.int32(10000)
x = numpy.arange(num_components, dtype=numpy.int32)

In [21]:
x_gpu = pycuda.gpuarray.to_gpu(x)
y_gpu = pycuda.gpuarray.zeros(num_components, dtype=numpy.int32)

In [22]:
threads_per_block = (256, 1, 1)
blocks_per_grid = (math.ceil(num_components / threads_per_block[0]), 1, 1)

In [23]:
plus_one_kernel(num_components, y_gpu, x_gpu, block=threads_per_block, grid=blocks_per_grid)

In [24]:
# Measure CPU time
time_start_cpu = time.time()
x = x + 1
time_end_cpu = time.time()

In [25]:
print("CPU calculation {0} [msec]".format(1000 * (time_end_cpu - time_start_cpu)))

CPU calculation 0.09751319885253906 [msec]


In [26]:
time_start_gpu.record()
plus_one_kernel(num_components, y_gpu, x_gpu, block=threads_per_block, grid=blocks_per_grid)
time_end_gpu.record()
time_end_gpu.synchronize()

In [27]:
print("GPU calculation {0} [msec]".format(time_start_gpu.time_till(time_end_gpu)))

GPU calculation 0.12636800110340118 [msec]


In [28]:
num_components = numpy.int32(10000000)
x = numpy.arange(num_components, dtype=numpy.int32)

In [29]:
x_gpu = pycuda.gpuarray.to_gpu(x)
y_gpu = pycuda.gpuarray.zeros(num_components, dtype=numpy.int32)

In [30]:
threads_per_block = (256, 1, 1)
blocks_per_grid = (math.ceil(num_components / threads_per_block[0]), 1, 1)

In [31]:
plus_one_kernel(num_components, y_gpu, x_gpu, block=threads_per_block, grid=blocks_per_grid)

In [32]:
# Measure CPU time
time_start_cpu = time.time()
x = x + 1
time_end_cpu = time.time()

In [33]:
print("CPU calculation {0} [msec]".format(1000 * (time_end_cpu - time_start_cpu)))

CPU calculation 16.999006271362305 [msec]


In [34]:
time_start_gpu.record()
plus_one_kernel(num_components, y_gpu, x_gpu, block=threads_per_block, grid=blocks_per_grid)
time_end_gpu.record()
time_end_gpu.synchronize()

In [35]:
print("GPU calculation {0} [msec]".format(time_start_gpu.time_till(time_end_gpu)))

GPU calculation 0.49670401215553284 [msec]


In [36]:
time_start_gpu.record()
# plus_one_kernel(num_components, y_gpu, x_gpu, block=threads_per_block, grid=blocks_per_grid)
y_gpu.get()
time_end_gpu.record()
time_end_gpu.synchronize()


In [37]:
print("GPU calculation {0} [msec]".format(time_start_gpu.time_till(time_end_gpu)))

GPU calculation 15.400383949279785 [msec]
